## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Tomas
- Apellido: Neustadt
- Legajo: 1645640

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
# Verificamos si estamos en Colab
var_google_colab = 'google.colab' in str(get_ipython())
print(var_google_colab)
if var_google_colab:
  #Montamos nuestro G.Drive
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)

True
Mounted at /content/gdrive


In [4]:
if var_google_colab:
  # Direccion root donde está la jupyter-notebook
  root_path = "/content/gdrive/MyDrive/tp-2/TP2_airbnb_london.csv"
  # Direccion donde guardaremos las imagenes
  plot_path = root_path
else:
  # Si, no estamos en google colab, es que estamos corriendo la 
  # en local.
  root_path = ""
  plot_path = root_path

In [5]:
london = pd.read_csv(root_path)

In [6]:
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [7]:
london.shape

(51827, 9)

In [8]:
london = london.iloc[:, 1:9]

In [9]:
london.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Haringey,Private room,35,1,131,1.71,4,262
1,Ealing,Private room,70,2,2,0.07,6,62
2,Islington,Private room,45,1,14,0.14,2,364
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [10]:
london.isnull().any()

neighbourhood                     False
room_type                         False
price                             False
minimum_nights                    False
number_of_reviews                 False
reviews_per_month                 False
calculated_host_listings_count    False
availability_365                  False
dtype: bool

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [11]:
london['neighbourhood'].dtypes

dtype('O')

In [12]:
london['room_type'].dtypes

dtype('O')

##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [13]:
## Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
neigh_dummies = pd.get_dummies(london['neighbourhood'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [14]:
neigh_dummies.shape

(51827, 33)

In [15]:
## Genero las dummies para la variable room_type, dentro de la variable room_dummies.
room_dummies = pd.get_dummies(london['room_type'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [16]:
room_dummies.shape

(51827, 3)

In [17]:
room_dummies.head()

,Entire home/apt,Private room,Shared room
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


Joinear las variables dummies al dataset



In [18]:
london = pd.concat([london, room_dummies,neigh_dummies],axis=1)

In [19]:
london = london.iloc[0:5000]

##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [20]:
y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [21]:
xtrain, xtest, ytrain, ytest = train_test_split(x.values, y, test_size=0.3, random_state=4)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [22]:
scaler = preprocessing.StandardScaler().fit(xtrain)

In [23]:
xtrain_scal = scaler.transform(xtrain)

In [24]:
xtest_scal = scaler.transform(xtest)

In [25]:
xtrain_sd = xtrain[:, 0:5] #sin dummies

In [26]:
xtrain_cd = xtrain[:, 5:] #con dummies

In [27]:
xtest_sd = xtest[:, 0:5] #sin dummies


In [28]:
xtest_cd = xtest[:, 5:] #con dummies

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [29]:
def lin_reg(x,y):
  n, m = x.shape
  I = np.identity(m)
  w = np.dot(np.dot(np.linalg.inv(np.dot(x.T, x)), x.T), y)
  return w

In [30]:
weights_ols = lin_reg(xtrain_scal,ytrain)
weights_ols

array([[   5.23106974],
       [  -3.92099972],
       [  -0.48896276],
       [   2.60447277],
       [   7.28114259],
       [ -86.966721  ],
       [-107.37185971],
       [ -16.36606931],
       [  -3.09486878],
       [ -14.87051184],
       [  -3.82394794],
       [ -19.25148052],
       [  -6.80895931],
       [ -40.61550327],
       [  -4.47957702],
       [  -3.98439936],
       [ -14.35500449],
       [ -14.60844259],
       [ -12.96384852],
       [ -35.68740556],
       [ -29.07937926],
       [ -20.2106182 ],
       [  -6.42759144],
       [  -2.47789732],
       [  -3.61690912],
       [  -7.19718412],
       [  -6.82740592],
       [   2.44367546],
       [  -2.91755329],
       [ -38.81812991],
       [ -19.61075302],
       [  -2.80187839],
       [ -12.48342309],
       [ -15.31591395],
       [ -15.22558312],
       [ -48.74368199],
       [  -5.29479143],
       [ -19.70870042],
       [ -16.3535929 ],
       [ -12.61366555],
       [  -9.09465706]])

In [31]:
np.linalg.norm(weights_ols, ord = 2)

175.84722236544522

In [32]:
y_pred_val_ols = np.dot(xtest_scal, np.asarray(weights_ols))

In [33]:
rmse_ols = np.sqrt(np.mean(np.square(ytest - y_pred_val_ols)))
rmse_ols

108.30534854110087

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [34]:
svreg = SVR()

In [35]:
param_svreg = {'kernel':('linear', 'rbf'), 'C':[1, 10,100 ], 'gamma':[ 0.1,1,10]}

In [36]:
regressor_svr = GridSearchCV(svreg , param_svreg, cv = 5, n_jobs = -1, verbose = True, refit = True)

In [37]:
regressor_svr.fit(xtrain_scal, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1, 10, 100], 'gamma': [0.1, 1, 10],
                         'kernel': ('linear', 'rbf')},
             verbose=True)

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [38]:
regressor_svr.best_params_

{'C': 100, 'gamma': 0.1, 'kernel': 'linear'}

In [39]:
regressor_svr.best_score_

0.47254978018813637

In [40]:
ypred = regressor_svr.predict(xtest_scal)
ypred

array([ 43.10170062,  50.29888081, 123.78289627, ...,  69.15203165,
       109.16629662, 129.30539078])

In [41]:
np.sqrt(mean_squared_error(ytest, ypred))

38.68056074825764

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [42]:
svreg = SVR()

In [43]:
param_svreg = {'kernel':('linear', 'rbf'), 'C':[80, 100, 120], 'gamma':[0.05, 0.1, 0.15]}

In [44]:
regressor_svr = GridSearchCV(svreg , param_svreg, cv = 5, n_jobs = -1, verbose = True, refit = True)

In [45]:
regressor_svr.fit(xtrain_scal, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [80, 100, 120], 'gamma': [0.05, 0.1, 0.15],
                         'kernel': ('linear', 'rbf')},
             verbose=True)

In [46]:
regressor_svr.best_params_

{'C': 80, 'gamma': 0.05, 'kernel': 'rbf'}

In [47]:
regressor_svr.best_score_

0.48292220595858276

In [48]:
ypred = regressor_svr.predict(xtest_scal)
ypred

array([ 37.76474307,  41.85796266, 100.01901965, ...,  61.73402886,
        96.17260024, 132.76399381])

In [49]:
np.sqrt(mean_squared_error(ytest, ypred))

38.450759208804485